# Set-up

In [1]:
!pip install tetgen
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay
!pip install pygmsh
!pip install tqdm
!pip uninstall -y h5py
!pip install h5py==2.9.0

     |████████████████████████████████| 1.9 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 88.3 MB 4.9 kB/s eta 0:00:01    |▊                               | 1.8 MB 7.7 MB/s eta 0:00:12     |██▎                             | 6.3 MB 7.7 MB/s eta 0:00:11     |██▌                             | 7.0 MB 3.8 MB/s eta 0:00:22     |████▋                           | 12.6 MB 3.8 MB/s eta 0:00:20     |████▊                           | 12.9 MB 3.8 MB/s eta 0:00:20     |████████████▍                   | 34.2 MB 6.3 MB/s eta 0:00:09     |██████████████▉                 | 41.1 MB 8.8 MB/s eta 0:00:06     |████████████████▌               | 45.6 MB 8.8 MB/s eta 0:00:05     |████████████████████▋           | 56.9 MB 4.1 MB/s eta 0:00:08     |█████████████████████▎          | 58.8 MB 4.1 MB/s eta 0:00:08     |██████████████████████          | 60.6 MB 5.9 

In [77]:
# Mesh imports:
import gmsh
import pygmsh
import meshio
import tetgen

# Visualisation imports:
import pyvista

# Misc imports:
from math import sin, cos, pi, ceil, floor
import os

tet2hex code:

In [78]:
from itertools import combinations
import numpy as np
from tqdm import tqdm

NUM_VERT=4
NUM_SUBDIV=4
EDGE_COMBOS = list(combinations(range(NUM_VERT),2))
FACE_COMBOS = list(combinations(range(NUM_VERT),3))
EPS = 1e-12
ORDERING_AXIS = 2


EL_0_ORDER = ((  (0,),   (0,1),   (0,1,2),   (0,2)),
               ((0,3), (0,1,3), (0,1,2,3), (0,2,3)))
EL_1_ORDER = ((  (1,),   (1,2),   (0,1,2),   (0,1)),
               ((1,3), (1,2,3), (0,1,2,3), (0,1,3)))
EL_2_ORDER = (( (2,),   (0,2),   (0,1,2),   (1,2)),
              ((2,3), (0,2,3), (0,1,2,3), (1,2,3)))
EL_3_ORDER = (((0,3), (0,1,3), (0,1,2,3), (0,2,3)),
              ( (3,),   (1,3),   (1,2,3),   (2,3)))
SUBDIV_ELS = (EL_0_ORDER, EL_1_ORDER, EL_2_ORDER, EL_3_ORDER)

SUBDIV_VERTS = (*[(i,) for i in range(NUM_VERT)],
                *EDGE_COMBOS,
                *FACE_COMBOS,
                tuple(i for i in range(NUM_VERT)))

SUBDIV_ORDER = []
for el in SUBDIV_ELS:
    face_idx = []
    for f in el:
        vert_idx = []
        for v_1 in f:
            vert_i = [idx for idx, v_2 in enumerate(SUBDIV_VERTS) if v_1==v_2]
            vert_idx.append(vert_i[0])
        face_idx.append(tuple(vert_idx))
    SUBDIV_ORDER.append(tuple(face_idx))
SUBDIV_ORDER = tuple(SUBDIV_ORDER) 

def tet2hex(mesh):
    old_coords = mesh.points
    new_cells, new_coords = np.empty((0,8), int), np.empty((0,3), float)
    for i, verts in enumerate(tqdm(mesh.cells[0].data)):
        # Compute coordinates of four hexahedra which will formed 
        # by dividing up this tetrahedron:
        subdiv_coords = compute_subdiv_coords(verts, old_coords)
        # Update the coordiantes list and cells list:
        new_cells, new_coords = update_cells_and_coords(subdiv_coords, new_cells, new_coords)
    
    new_mesh = meshio.Mesh(new_coords, [("hexahedron", new_cells)])
    return new_mesh

def compute_subdiv_coords(verts, coords):
    vert_coords = np.array([coords[v] for v in verts])
    
    # Order vertex along z axis:
    ordering = np.argsort(vert_coords[:, ORDERING_AXIS])
    vert_coords = vert_coords[ordering]
    
    edge_centres = [np.mean([vert_coords[idx] for idx in edge], axis=0) for edge in EDGE_COMBOS]
    face_centres = [np.mean([vert_coords[idx] for idx in faces], axis=0) for faces in FACE_COMBOS]
    vol_centre = np.mean(vert_coords, axis=0)
    subdiv_coords = np.vstack([*vert_coords, *edge_centres, *face_centres, vol_centre])
    return subdiv_coords

def update_cells_and_coords(hex_coords, new_cells, new_coords):
    
    # First, see if any hex coordinates already exist within our mesh:
    exist_pts = find_exist_pts(hex_coords, new_coords)
    
    # Create new coordinates for those points not currently in the mesh:
    notexist_pts = {}
    num_coords = len(new_coords)
    for i, c in enumerate(hex_coords):
        if i not in exist_pts.keys():
            # Number this new point:
            notexist_pts[i] = num_coords
            # Add new coordinates to coordinates list:
            new_coords = np.vstack([new_coords, c])
            # Note we have one more point in mesh:
            num_coords += 1
    
    global_idx = exist_pts
    global_idx.update(notexist_pts)
    for elem in range(NUM_SUBDIV):
        cell_i = []
        for face in (0, 1):
            cell_i += [global_idx[i] for i in SUBDIV_ORDER[elem][face]]
        new_cells = np.vstack([new_cells, cell_i]) 
    return (new_cells, new_coords)

def find_exist_pts(hex_coords, coords):
    exist_pts = {}
    if coords.size>0:
        for i, c in enumerate(hex_coords):
            coord_dist = abs(coords - c)
            pts_same = np.all(coord_dist<EPS, axis=1)
            if pts_same.sum():
                exist_pts[i] = np.where(pts_same)[0].item()
    return exist_pts

def order_cell_verts(el_idx, cell_i):
    for face in (0, 1):    
        ordering = SUBDIV_ORDER[el_idx][face]

Functions to convert between Meshio and Pyvista:

In [79]:
def meshio_to_pyvista(mesh):
    meshio_mesh = meshio.Mesh(mesh.points, mesh.cells_dict)
    meshio_mesh.write('temp.msh')
    pyvista_mesh = pyvista.read('temp.msh')
    os.remove('temp.msh')
    return pyvista_mesh

def pyvista_to_meshio(mesh):
    cells = list(mesh.cells_dict.values())[0]
    cell_type = 'tetra' if cells.shape[1]==4 else 'hexahedron'
    points = mesh.points
    meshio_mesh = meshio.Mesh(points, {cell_type:cells})
    return meshio_mesh
    
# def meshio_to_grid(meshio_mesh):
#     topology, cell_types = dolfinx.plot.create_vtk_topology(meshio_mesh, meshio_mesh.topology.dim)
#     grid = pyvista.UnstructuredGrid(topology, cell_types, meshio_mesh.geometry.x)
#     return grid

Visualisation functions:

In [80]:
def visualise_mesh(grid, subgrid=None, title=None):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    if subgrid is not None:
        p.add_mesh(grid, style="wireframe", color="k") 
        p.add_mesh(subgrid, color='white', lighting=True, show_edges=True) 
    else: 
        p.add_mesh(grid, show_edges=True, edge_color='k', color='white', lighting=False) # 
    
    p.show_axes()
    # p.show_bounds()
    p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

# Function to plot specific of hex elements of tetrahedron:
def plot_single_tet(hex_mesh, elements_to_show):
    coords = hex_mesh.points
    cells = hex_mesh.cells[0].data[elements_to_show]
    new_hex = meshio.Mesh(coords, [("hexahedron", cells)])
    viewer = visualise_mesh(new_hex)
    return viewer

def get_submesh(pyvista_mesh, axis, cutoff):
    cell_center = pyvista_mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    submesh = pyvista_mesh.extract_cells(cell_ind)
    return submesh

Function to 'solidify' surface mesh:

In [81]:
def create_volume_mesh(pyvista_mesh, mindihedral=1):
    tetgen_mesh = tetgen.TetGen(pyvista_mesh)
    tetgen_mesh.tetrahedralize(order=1, mindihedral=mindihedral) 
    grid = tetgen_mesh.grid
    grid_cells = grid.cells_dict.pop(10)
    meshio_mesh = meshio.Mesh(grid.points, [("tetra", grid_cells)])
    return meshio_mesh

# Single Tetrahedron Example

In [82]:
tet_coords = np.array([[0, 0, 0],
                       [1, 0, 0],
                       [0, 1, 0],
                       [0, 0, 1]])
tet_cells = np.array([[0,1,2,3]])
tet_mesh = meshio.Mesh(tet_coords, [("tetra", tet_cells)])

In [83]:
visualise_mesh(tet_mesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [84]:
hex_mesh = tet2hex(tet_mesh)

100%|██████████| 1/1 [00:00<00:00, 480.23it/s]


In [85]:
elements_to_show = [0,1,2,3] #
plot_single_tet(hex_mesh, elements_to_show)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

# Breast Mesh Example

In [86]:
# Load mesh:
tet_mesh = pyvista.read('breast_hex.obj')
                       
# Create solid tetrahedral mesh from surface mesh:
tet_mesh = create_volume_mesh(tet_mesh)

In [87]:
# Original tet mesh:
visualise_mesh(tet_mesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [92]:
tet_mesh_pv

UnstructuredGrid (0x7f58300beee0)
  N Cells:	481
  N Points:	200
  X Bounds:	7.137e-01, 9.533e+01
  Y Bounds:	5.395e+00, 1.372e+02
  Z Bounds:	7.578e+00, 1.975e+02
  N Arrays:	4

In [88]:
# Original tet with slice:
axis=1
cutoff=75
tet_mesh_pv = meshio_to_pyvista(tet_mesh)
submesh = get_submesh(tet_mesh_pv, axis, cutoff)
visualise_mesh(tet_mesh_pv, submesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [89]:
# Convert to hexahedral volume mesh:
hex_mesh = tet2hex(tet_mesh)

# Convert hex mesh to pyvista format:
hex_mesh = meshio_to_pyvista(hex_mesh)

100%|██████████| 481/481 [00:00<00:00, 685.30it/s]


In [91]:
hex_mesh

UnstructuredGrid (0x7f58300ea880)
  N Cells:	1924
  N Points:	2713
  X Bounds:	7.137e-01, 9.533e+01
  Y Bounds:	5.395e+00, 1.372e+02
  Z Bounds:	7.578e+00, 1.975e+02
  N Arrays:	2

In [90]:
# New hex mesh:
visualise_mesh(hex_mesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [76]:
# Hex mesh with slice:
axis=1
cutoff=75
submesh = get_submesh(hex_mesh, axis, cutoff)
visualise_mesh(hex_mesh, submesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)